In [1]:
import os
import json
import pandas as pd
from pathlib import Path
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
)
from IPython.display import display, Markdown
import torch
import aisuite as ai
from IPython.display import display, Markdown
import pandas as pd
from pathlib import Path
import json
import os


# Paths
paths = {
    'root': Path.cwd().parent,
    'data': Path.cwd().parent / "data",
    "config": Path.cwd().parent / "config"
}

# Load Hugging Face credentials
with open(paths["config"] / 'credentials.json') as f:
    credentials = json.load(f)


if "HUGGINGFACE_TOKEN" in os.environ or "HUGGINGFACE_TOKEN" in credentials:
    print("Environment variable HUGGINGFACE_TOKEN set.")


# Define the file path
file_path = paths["config"] / "instructions.txt"

try:
    # Open the file and read its content
    with open(file_path, 'r') as file:
        instructions = file.read()
        print("Instructions successfully read!")
        # print(instructions)
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


# Shortlisted candidates
try:
    list_of_candidates = pd.read_parquet(paths['data'] / "processed/filtered.parquet", columns=['job_title', 'rank']).set_index("rank")['job_title'].to_list()
except Exception as e:
    print(f"Failed to load parquet file: {e}. Loading CSV instead.")
    list_of_candidates = pd.read_csv(paths['data'] / "processed/filtered.csv", index_col=['rank'], usecols=['job_title'])['job_title'].to_list()

c:\Users\Guill\Miniconda3\envs\a4_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Environment variable HUGGINGFACE_TOKEN set.
Instructions successfully read!


In [2]:
# Initialize the model and tokenizer
model_name = "microsoft/Phi-3-mini-128k-instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

# Adjust model loading for GPU
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    trust_remote_code=True,
    device_map=device,  # Use GPU if available
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set up the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.18s/it]
Device set to use cuda


In [13]:
list_similar_roles = 'aspiring human resources'
location = "New York"

search_criteria = """**Criteria:**
- **Motivation:** Demonstrated interest and passion for a career in Human Resources.
- **Experience & Background:** Relevant experience, transferable skills, and educational background.
- **Fit with Company Values:** Highly motivated individuals with fundamental HR knowledge are preferred over those with extensive credentials.
"""

response_format = """**Response Format:**
Provide a ranked table in Markdown format with the following columns:
- **Rank:** Position from 1 (top) to 5.
- **Candidate:** Description of the candidate.
- **Experience & Background:** Summary of relevant experience and qualifications.
- **Reasoning:** Explanation for the ranking based on the criteria.
"""

user_prompt = f"""**Prompt:**
Valuate the list of candidates based on the search criteria and location: "{location}" for the role of {list_similar_roles}.

{search_criteria}

Identify and rank the top 5 candidates who are most suitable for a position in human resources. Present your findings in a Markdown-formatted table that includes the **Candidate**, **Experience & Background**, and **Reasoning for Choosing This Candidate**.

**List of Candidates:**
{list_of_candidates}
"""

role_system_description = (
    "You are an HR Recruitment Specialist. Your role is to assist in identifying, evaluating, "
    "and ranking job candidates based on their qualifications, experience, and suitability for specific roles. "
    "You prioritize motivated individuals with fundamental knowledge and present information in clear, organized formats to support hiring decisions."
)


# Prepare messages in the format required by the model
messages = [
    {"role": "system", "content": role_system_description},
    {"role": "user", "content": user_prompt},
]

# Generate text
generation_args = {
    "max_new_tokens": 5_000,
    "return_full_text": False,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
display(Markdown("Generated Output:", output[0]['generated_text']))

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.
